# <center>Частина І</center>

Функція корисності: $U(x) = 3.3(x_1 - 8)^{0.48} + 5.8(x_2 - 11)^{0.14} + 15.2x_3^{0.64} + 9(x_4 - 4)^{0.43}$

In [1]:
def U(x1, x2, x3, x4):
    return (3.3*(x1 - 8) ** 0.48) + (5.8*(x2 - 11) ** 0.14 ) + (15.2*x3 ** 0.64) + 9*(x4 - 4) ** 0.43

## Маршаліанська поведінка споживача

**Ціль задачі:** максимізація корисності при заданих доходах, **Обмеження:** витрати не перевищують дохід 

In [4]:
# prices
p = [20, 25, 10, 15]
 
I = 1000

def objective_marshall(x):
    return -U(x[0], x[1], x[2], x[3])

def constraint_marshall(x):
    return I - x[0] * p[0] - x[1] * p[1] - x[2] * p[2] - x[3] * p[3]

Максимізуємо корисність при заданих доходах та певних обмеженнях

In [6]:
import numpy as np
from scipy.optimize import minimize 

x0 = [15, 15, 15, 15]
 
bounds = (
    (8, np.inf),
    (11, np.inf),
    (0, np.inf),
    (4, np.inf)
)

solution = minimize(objective_marshall, x0, method='SLSQP', bounds=bounds, constraints=[{'type': 'ineq', 'fun': constraint_marshall}])
 
x = solution.x
 
print('Максимальна корисність: ' + str(-objective_marshall(x)))
 
print('Розв\'язок')
x

Максимальна корисність: 196.706302156
Розв'язок


array([  8.11798357,  11.09737005,  48.32798163,   5.1284174 ])

Знаходимо множники Лагранжа як відношення часткової похідної до ціни:

In [4]:
from scipy.misc import derivative
 
def partial_derivative(func, var=0, point=[]):
    args = point[:]
 
    def wraps(x):
        args[var] = x
        return func(*args)
 
    return derivative(wraps, point[var], dx = 1e-6)

print("Множники Лагранжа")
L = [partial_derivative(U, i, x) / p[i] for i in range(4)]
for i in range(4):
    print('L{} = {}'.format(i + 1, L[i]))

Множники Лагранжа
L1 = 0.24064562893499897
L2 = 0.24075214639651676
L3 = 0.24083080774062182
L4 = 0.24083074852872718


Отже, значенням маргінальної корисності доходу споживача є 0.24

## Хіксіанівська поведінка споживача

**Ціль задачі:** а мінімальну ціну знайти корисність не менше заданої, **Обмеження на функцію:** функція має бути більша ніж добробут(заданий) 

In [5]:
def objective_hicks(h):
    return p[0] * h[0] + p[1] * h[1] + p[2] * h[2] + p[3] * h[3]

def constraint_hicks(h):
    return U(h[0], h[1], h[2], h[3]) - U(9, 12, 0, 4)

Мінімізуємо функцію з обмеженнями:

In [6]:
h0 = [20, 20, 20, 20]

solution = minimize(objective_hicks, h0, method='SLSQP', bounds=bounds, constraints=[{'type': 'ineq', 'fun': constraint_hicks}])
 
h = solution.x
 
print('Мінімальні витрати: ' + str(objective_hicks(h)))
 
print('Розв\'язок')
for i in range(4):
    print('h{} = {}'.format(i + 1, h[i]))
print('U(h) = {}'.format(U(h[0], h[1], h[2], h[3])))
 


Мінімальні витрати: 496.9250711570113
Розв'язок
h1 = 8.00194029443267
h2 = 11.008128334536874
h3 = 0.128339864465244
h4 = 4.026643884018906
U(h) = 9.09999990197845


Знаходимо множники Лагранжа як відношення ціни до часткової похідної:

In [7]:
print("Множники Лагранжа")
L = [p[i] / partial_derivative(U, i, h) for i in range(4)]
for i in range(4):
    print('L{} = {}'.format(i + 1, L[i]))

Множники Лагранжа
L1 = 0.4908694173209781
L2 = 0.4908912915885567
L3 = 0.49089273582899473
L4 = 0.4908749709190122


Отже, значенням додаткових витрат при зміні вартості є 0.49

# <center>Частина ІI</center>

Будуємо мультиплікативну функцію: $$ F(K,L) = a * K^b * L^c $$ на даних:

In [10]:
# Kапітал
K = [10600, 10550, 10630, 10790, 10860, 10380, 10630, 10600, 10800, 10740]

In [11]:
# Кількість продукту / обсяг виробництва
F = [25071, 25064, 25226, 25461, 25935, 24700, 25015, 25018, 25626, 25580]

In [12]:
# Праця
L = [5070, 5000, 5000, 5150, 5260, 4950, 5015, 5020, 5170, 5195]

In [17]:
def mult_func(x, a, b, c):
    return a * (x[0] ** b) * (x[1] ** c) 

Знаходимо коефіцієнти:

In [18]:
from scipy.optimize import curve_fit

p0 = [6.7670, 0.3580, 0.3629]
coeffs, _ = curve_fit(mult_func, (K, L) , F, p0)
print("Виробнича функція\n F = {} * K ^ {} * L ^ {}".format(coeffs[0], coeffs[1], coeffs[2]), "\n")

Виробнича функція
 F = 7.9995573693940925 * K ^ 0.525421003214771 * L ^ 0.3732516730550991 



Знаходимо ефект маштабу порівнюючи суму знайдених коефіцієнтів:


In [31]:
k = 2
arg_inc = mult_func(k * (K, L), *coeffs)
func_inc = k* mult_func((K, L), *coeffs)
stable = incr = dect = False
for i in range(len(K)):
    if  arg_inc[i] == func_inc[i]:
        stable = True
    elif arg_inc[i] > func_inc[i]:
        incr = True
    elif arg_inc[i] < func_inc[i]:
        decr = True
print ("Стабільна: " + str(stable))
print ("Спадаюча: " + str(decr))
print ("Зростаюча: " + str(incr))

Стабільна: False
Спадаюча: True
Зростаюча: False


Можна довести, що еластичність заміщення для мультиплікативної ф-ції завжди рівна 1

## Повна конкуренція

Не можемо впливати на ціну, бо її диктує ринок.    

In [14]:
# ціна товара
price = 7
# ціна на капітал, працю
w = [10, 20]

Існує два періода:
* короктостроковий - присутні обмеження на ресурси
* довгостроковий - немає обмежень на ресурси

### Довгостроковий період

In [15]:
#Формула 
def pi(x):
    return w[0] * x[0] + w[1] * x[1] - price * mult_func(x, *coeffs)

Обмеження на ресурси відсутні

In [16]:
bounds = (
    (0, np.inf),
    (0, np.inf)
)

Максимізуємо функцію прибутку/ прибуток:

In [17]:
long_solution = minimize(pi, [1, 1], method='SLSQP', bounds=bounds, constraints=[])
print(long_solution.x)
print("Довгостроковий прибуток:", -pi(long_solution.x), "\n")

[931.98617312 331.02647392]
Довгостроковий прибуток: 1797.2995338043202 



### Короткостроковий період

Обмеження:

In [18]:
def constraint(x):  # sqrt(x[0] * x[0] + x[1] * x[1]) <= 5000
    return 5 - (x[0] * x[0] + x[1] * x[1]) ** 0.5

Максимізуємо функцію прибутку тепер вже з обмеженнями:

In [19]:
short_solution = minimize(pi, [1, 1], method='SLSQP', bounds=bounds, constraints=[{'type': 'ineq', 'fun': constraint}])
print(short_solution.x)
print("Короткостроковий прибуток:", -pi(short_solution.x))

[4.36004218 2.44745424]
Короткостроковий прибуток: 76.9834769149318


## Монополія. Монопсонія

* **Монополія** - єдиний продавець 
* **Монопсонія** - єдиний покупець

Ціни - це функції.
Максимізуємо чистий прибуток з урахуванням функції цін.

Функція попиту:

In [20]:
def price_func(x): 
    return -x / 10 + 1000

Функція пропозиції:

In [21]:
def wL(x):
    return 0.2 * x[1] - 10

Капітал:

In [22]:
def wK(x):
    return 0.1 * x[0] - 20

Фінальні розв'язки:

In [1]:
def monopoly_pi(x):
    Q = mult_func(x, *coeffs)
    return wK(x) * x[0] + wL(x) * x[1] - Q * price_func(Q)
 
 
monopoly_solution = minimize(monopoly_pi, [1, 1], method='SLSQP', bounds=bounds, constraints=[])
print(monopoly_solution.x)
print("Прибуток монополії:", -monopoly_pi(monopoly_solution.x))
print("Ціна на товар:", price_func(mult_func(monopoly_solution.x, *coeffs)))
print("Ціна ресурсів: K = {}, L = {}".format(wK(monopoly_solution.x), wL(monopoly_solution.x)))
print("Обсяг виробництва:", mult_func(monopoly_solution.x, *coeffs))

NameError: name 'minimize' is not defined